In [ ]:
#########################################################
####
#### Example: RNA Multi-target Design with Infrared
####
#########################################################

In [ ]:
import infrared as ir
from infrared import rna
import RNA

##################################################
## our target RNA secondary structure
#             01234567890123456789
structures = list()
structures.append( "((((...)))).(((...)))" )
structures.append( "((((((......)))...)))" )
#structures.append( "......(((...)))......" )

seqlen = len(structures[0])

######
# construct the constraint model
model = ir.Model()

# one variable X_i per position i;
# the value of X_i encodes the nucleotide at position i   
model.add_variables( seqlen, 4 )


for i,structure in enumerate(structures):
    bps = rna.parse_RNA(structure)

    model.add_constraints( rna.ComplConstraint( i, j ) for ( i, j ) in bps )
    
    model.add_functions( [ rna.BPEnergy( i, j, False ) 
                           for ( i, j ) in bps ], group = f'bpenergy{i}' )
    
    model.add_feature( f'E{i}', # feature name
                       f'bpenergy{i}', # controlled group(s)
                       #
                       # function to evaluate the feature for a sample;
                       # NOTE how we have to bind i
                       lambda sample, i=i: RNA.energy_of_struct( rna.values_to_seq( sample.values() ),
                                              structures[i] )
                     )

model.add_functions( [ rna.GCControl( i = i ) for i in range(seqlen) ], group = 'gc' )

# the model generates automatic features 'bpenergyI', 'gc' from the function groups;
# as well as total feature combining all function groups;
# however, we want to diretly control Turner energy (instead of base pair energy).
# For this purpose, add additional features 'EI'


def print_sample(sample):    
    seq = rna.values_to_seq( sample.values() )
    
    print("{} GC={:.2f}".format( seq,
                              model.eval_feature(sample, 'gc')*100/seqlen ),end=""
         )
    for i,s in enumerate(structures):
        print(f" E{i}={model.eval_feature(sample, f'E{i}'):.2f}",end="")
    print()
    


print("###########################################")    
## Sampling at specific weights

sampler = ir.BoltzmannSampler( model )

print( "Tree width:", sampler.treewidth() )

sampler.plot_td("treedecomp.pdf")

######
# set targets

model.set_feature_weight( -5, 'E0' )
model.set_feature_weight( -2, 'gc' )

######
# and draw samples
for i in range(10):
    sample = sampler.sample()
    print_sample(sample)
    
    
print("###########################################")    
## MDBS

model.set_feature_weight( 0, 'E0' )
model.set_feature_weight( 0, 'E1' )
#model.set_feature_weight( 0, 'E2' )
model.set_feature_weight( 0, 'gc' )

######
# create sampler
sampler = ir.Sampler( model )

######
# set targets

# control number of gc's; we target 70% +/- 15% GC-content
sampler.set_target( 0.85 * seqlen, 0.02 * seqlen, 'gc' )

# control Turner energy, target -2 +/- 1 kcal/mol
sampler.set_target( -2, 0.2, 'E0' )

# control Turner energy, target -2 +/- 1 kcal/mol
sampler.set_target( -3, 0.2, 'E1' )

# control Turner energy, target -2 +/- 1 kcal/mol
#sampler.set_target( -1.5, 0.2, 'E2' )

######
# and draw samples
for i in range(10):
    sample = sampler.targeted_sample()
    print_sample(sample)